## Imports

In [25]:
%%capture
%run 'Distributions.ipynb'

import numpy as np
import pandas as pd

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [35]:
class Sample(object):
    """ The sample data """
    
    def __init__(self,x,y,name):
        self.x = x
        self.y = y
        self.name = name
    
    @classmethod
    def from_distibution(cls, x, N, distribution, name=''):
        
        return cls(x, (N*distribution.get_pdf_values(x)).astype(int),name)
    
    def get_pdf(self):
        return self.y
    
    def plot_sample(self):
        iplot(go.Figure(
            data=[go.Bar(x=self.x,y=self.get_pdf())],
            layout=go.Layout(title=self.name)))

In [37]:
s1 = Sample.from_distibution(np.arange(0,10,0.2), 100,
                             BoundedNormalDistribution(mean=1, stddev=1), 
                             name='Poitive Sample')

s1.plot_sample()

In [10]:
class SliderPlotHandler(object):
    """ Handles the ROC plots """
    
    def __init__(self,plot_range, plot_type=go.Bar, **params):
        
        self.plot_type = plot_type
        self.plot_range = plot_range
        self.data = []
        for k,v in params.items():
            setattr(self,k,v)
        
    def process_sample(self,sample,color='00CED1'):
        
        return self.plot_type(
            visible = False, 
            marker=dict(color=color), 
            name = sample.name,
            x = sample.x, 
            y = sample.y)
    
    def set_sliding_sample(self, samples):
        for sample in samples:
            self.data.append(self.process_sample(sample))
    
    def set_static_sample(self,sample):
        self.data.append(self.process_sample(sample, color='FFCED1'))
    
    
    def get_slider_for_multiple_samples(self):
        
        steps = []
        
        for i in range(len(self.plot_range)):
            step = dict(
                method = 'restyle',
                args = ['visible', [False] * (len(self.data)-2)+[True,True]],
            )
            step['args'][1][i] = True
            steps.append(step)
        
        sliders = [dict(
            active = 10,
            pad = {"t": 50},
            steps = steps
        )]
        
        return sliders
            
    
    def get_roc_fig(self, positive_sample, negative_sample):
        
        classifier = [Sample.from_distibution(positive_sample.x, 
                                              DeltaDistribution(mean=i,stddev=0),name=str(i)) 
                      for i in positive_sample.x]
        
        self.set_sliding_sample(classifier)
        
        self.set_static_sample(negative_sample)
        
        self.set_static_sample(positive_sample)
        
#         sliders = self.get_slider_for_multiple_samples()
        sliders=[]
        return go.Figure(data=self.data, layout=go.Layout(sliders=sliders))
        

In [11]:
class ROCHandler(object):
    """ Handler of the ROC - part of the evaluation of the classifier. """
    
    def __init__(self, positive_sample, negative_sample):
        self.positive_sample = positive_sample
        self.negative_sample = negative_sample
        
    
    def plot_roc(self):
        
        plot_handler = SliderPlotHandler(plot_range=self.positive_sample.x)
        
#         return plot_handler.get_roc_fig(positive_sample=self.positive_sample, 
#                                        negative_sample=self.negative_sample)
        
        iplot(plot_handler.get_roc_fig(positive_sample=self.positive_sample, 
                                       negative_sample=self.negative_sample))

In [12]:
s1 = Sample.from_distibution(np.arange(0,10,0.2), 
                             BoundedNormalDistribution(mean=1, stddev=1), 
                             name='Poitive Sample')

s2 = Sample.from_distibution(np.arange(0,10,0.2), 
                             BoundedNormalDistribution(mean=4, stddev=3),
                             name='Negative Sample')

In [15]:
sph = SliderPlotHandler(plot_range=s1.x)

In [18]:
ps1 = sph.process_sample(s1)
ps1[ 'visible'] = True

In [19]:
iplot(go.Figure(data=[ps1]))

In [13]:
roc_handler = ROCHandler(s1,s2)

In [14]:
roc_handler.plot_roc()

In [ ]:
iplot(a)

In [ ]:
class SampleDistribution(object):
    
    def __init__(self, mean=0, std=1, plot_type=go.Bar,second_dist_mean=0.8,second_dist_std=1,classifier):
        self.pdf = bounded_gaussian
        self.mean = mean
        self.std = std
        self.second_dist_mean = second_dist_mean
        self.second_dist_std = second_dist_std
        self.plot_type = plot_type
        self.range = np.arange(0,10,0.2)
        
    def get_single_plot(self, step, color='00CED1',name='Positive Distribution'):
        return go.Bar(visible = False, 
                      marker=dict(color=color), 
                      name = name,
                      x = self.range, 
                      y = self.pdf(self.range, step, self.std))
    
    def get_slider_figure(self,slider_range=None):
        
        if slider_range == None:
            slider_range = self.range
        
        data = [self.get_single_plot(i) for i in slider_range]
        
        steps = []
        
        for i in range(len(slider_range)):
            step = dict(
                method = 'restyle',
                args = ['visible', [False] * len(slider_range)+[True]],
            )
            step['args'][1][i] = True # Toggle i'th trace to "visible"
            steps.append(step)
        
        sliders = [dict(
            active = 10,
            pad = {"t": 50},
            steps = steps
        )]
        

        second_dist = [self.get_single_plot(step=8, color='FFCED1',name='Negative Distribution')]
        
        layout = go.Layout(sliders=sliders)

        fig = go.Figure(data=data+second_dist, layout=layout)
        
        iplot(fig)
        

In [ ]:
dist=SampleDistribution()

In [ ]:
dist.get_slider_figure()

In [ ]:
data = [base_function(step) for step in np.arange(0,5,0.1)]
data[10]['visible'] = True

iplot(data)

In [ ]:
steps = []
for i in range(len(data)):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * len(data)],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = 10,
    currentvalue = {"prefix": "Frequency: "},
    pad = {"t": 50},
    steps = steps
)]

layout = dict(sliders=sliders)

fig = dict(data=data, layout=layout)

iplot(fig)